<a href="https://colab.research.google.com/github/21JN1A4215/FMML-MODULES/blob/main/Copy_of_Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [ ]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [ ]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [ ]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [ ]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [ ]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [ ]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


ANSWER:1

Yes, averaging the validation accuracy across multiple splits of your dataset typically gives more consistent and reliable results compared to relying on a single validation split. This technique is commonly known as "cross-validation."

Cross-validation involves splitting your dataset into multiple subsets or "folds" and training and validating your model on different combinations of these folds. By doing so, you can assess how well your model performs on different portions of your data. Averaging the validation accuracy across these different folds helps reduce the impact of randomness or variability that may occur when using a single validation split.

Cross-validation provides several benefits:

Reduced Variance: It helps you obtain a more stable estimate of your model's performance by averaging results across multiple splits, which reduces the impact of outliers or peculiarities in a single split.

Better Generalization: It provides a more accurate estimate of how well your model is likely to generalize to unseen data, as it evaluates the model's performance on multiple subsets of the data.

Fair Evaluation: It ensures that your model's performance is not overly influenced by the specific random choice of a single validation split.

Common types of cross-validation include k-fold cross-validation, leave-one-out cross-validation, and stratified cross-validation, each with its own advantages and trade-offs. The choice of which type to use depends on the nature of your data and the problem you are trying to solve.







ANSWER:2

Cross-validation, which involves averaging the validation accuracy across multiple splits of your dataset, can provide a more accurate estimate of how well your model is likely to perform on unseen data (i.e., test accuracy) compared to a single validation split. However, it's important to clarify the distinction between validation accuracy and test accuracy:

Validation Accuracy: This is used during the model development process to tune hyperparameters, select models, and assess how well your model is performing on a specific validation set. Cross-validation helps provide a more reliable estimate of this metric by averaging results across multiple validation splits.

Test Accuracy: This is the final evaluation metric used to assess how well your model generalizes to unseen data, which is typically a separate dataset that your model has never been exposed to during training or hyperparameter tuning.

Cross-validation does not directly estimate test accuracy; instead, it provides a more accurate estimate of how well your model is likely to generalize. To obtain a reliable estimate of test accuracy, you should reserve a separate test dataset that is not used during model development or hyperparameter tuning. The test dataset should represent real-world data your model will encounter.

You can use cross-validation to gain confidence in your model's expected performance, but ultimately, the test accuracy on a completely independent dataset is the most accurate reflection of how well your model will perform in practice.






ANSWER:3

The number of iterations (or epochs) in training a machine learning model can have an effect on the estimate of the model's performance, but it doesn't necessarily guarantee a better estimate with higher iterations. The impact of the number of iterations on your estimate depends on several factors, and there is a trade-off to consider:

Underfitting and Overfitting: Training a model for too few iterations can result in underfitting, where the model doesn't capture the underlying patterns in the data. Conversely, training for too many iterations can lead to overfitting, where the model starts to memorize the training data and performs poorly on unseen data. The "sweet spot" for the number of iterations is typically found somewhere in between, where the model generalizes well without overfitting.

Convergence: Many machine learning algorithms have a convergence point where they stop improving significantly. Once the model has converged, training it for additional iterations may not lead to substantial improvements in performance. In such cases, increasing the number of iterations beyond the convergence point may not provide a better estimate of the model's performance.

Computational Resources: Training for a higher number of iterations can be computationally expensive and time-consuming. You may need to balance the need for a better estimate with the available computational resources.

Early Stopping: To find the optimal number of iterations, many practitioners use techniques like early stopping, where training is halted when the validation performance plateaus or starts to degrade. This prevents overfitting and can lead to a more accurate estimate of the model's performance.

Regularization: Some regularization techniques, like dropout and weight decay, can help mitigate overfitting, allowing you to train models for more iterations without the risk of poor generalization.

In summary, the effect of the number of iterations on the estimate of your model's performance depends on the specific algorithm, dataset, and problem you are working on. It's essential to strike a balance between training long enough for the model to learn but not so long that it overfits. Techniques like cross-validation and early stopping can help you find an appropriate number of iterations for your model.






ANSWER:4

Increasing the number of iterations can help to some extent when dealing with a very small training dataset, but it may not completely overcome the limitations associated with a small dataset, especially when it comes to generalization and model performance. Here are some points to consider:

Overfitting: With a very small training dataset, there's a higher risk of overfitting. Overfitting occurs when a model learns to memorize the training data instead of generalizing from it. Increasing the number of iterations without addressing overfitting can make the problem worse.

Limited Diversity: Small datasets often lack the diversity and variety required for a model to learn robust and generalizable patterns. No matter how many iterations you perform, the model may struggle to capture the underlying data distribution.

Data Augmentation: Instead of solely relying on more iterations, consider data augmentation techniques. Data augmentation involves generating additional training examples by applying transformations (e.g., rotation, cropping, flipping) to the existing data. This can effectively increase the effective size of your training dataset and help improve model performance.

Regularization: Use regularization techniques like dropout, weight decay, or early stopping to prevent overfitting when working with small datasets. These techniques can help the model generalize better and avoid excessive reliance on the limited training data.

Transfer Learning: When you have a small dataset, leveraging pre-trained models through transfer learning can be highly effective. You can fine-tune a pre-trained model on your small dataset, benefiting from the knowledge learned on a larger, related dataset.

Model Complexity: Consider using simpler models with fewer parameters when working with small datasets. Complex models tend to overfit more easily, so a simpler architecture might be more appropriate.

In summary, while increasing the number of iterations can help when dealing with small datasets, it's only one part of the solution. Combining it with techniques like data augmentation, regularization, transfer learning, and careful model selection can lead to better performance and more reliable estimates of your model's capabilities.




